# Step 8  - Outgroup Testing
-- Alex Warwick Vesztrocy, February 2024

This notebook identifies relevant species to use for each branch in the species tree as outgroup species.

After, LDO / MDO are compared to the outgroup gene with both a PCC and tau analysis.

In [1]:
from collections import defaultdict
from dendropy import Tree
from scipy.stats import spearmanr, pearsonr
from tqdm.auto import tqdm
import numpy as np
import os
import pandas as pd

from parallel_pandas import ParallelPandas
ParallelPandas.initialize()

In [2]:
# species that have enough data and are in PANTHER
dataset = {'animals': {'BOVIN','CHICK','DROME','FELCA','GORGO','HORSE','HUMAN','LEPOC','MACMU','MONDO','MOUSE','ORNAN','ORYLA','PANTR','PIG','RAT'},
           'plants': {'AMBTC','ARATH','BRANA','CAPAN','CUCSA','HELAN','MAIZE','MANES','MARPO','MEDTR','ORYSJ','PHYPA','POPTR','SELML','SETIT','SOLLC','SOLTU','SOYBN','VITVI','WHEAT'}}

---
## Identify branches to study

For each branch in the tree, identify the outgroups and print the choices. We can then filter the outgroup test file to only the closest outgroup for each branch.

First, load the taxonomy used for PANTHER

In [3]:
t = Tree.get(path='./results/taxonomy_fit.nwk', schema='newick')
D = t.phylogenetic_distance_matrix()

In [4]:
def collect_out(df):
    outgroups = np.array(list(filter(lambda x: x.startswith('outgroup_ldo'), df.keys())))
    def gen():
        for (_, r) in df.iterrows():
            b = (r['sp_tree_tail'], r['sp_tree_head'])
            x = outgroups[r[outgroups].notna()]
            if len(x) > 0:
                yield (b,set(x))
    res = defaultdict(set)
    for x in gen():
        res[x[0]].update(set(map(lambda x: x.split('_')[-1], x[1])))
    return res

In [5]:
def generate_candidates(sp, pairs_df, possible_species=None):
    def gen():
        x = collect_out(pairs_df)
        sp_n = t.find_node_with_taxon_label(sp)
        for (b, cands) in x.items():
            y = []
            for c in cands:
                if possible_species is not None and c not in possible_species:
                    continue
                c_n = t.find_node_with_taxon_label(c)
                y.append((b, c, D.distance(sp_n.taxon, c_n.taxon)))
            if len(y) > 0:
                yield min(y, key=lambda x: x[2])
            
    return {x[0]: x[1] for x in gen()}

In [6]:
def filter_to_closest(df, closest_map):
    df = df.copy()
    outgroup_keys = list(filter(lambda x: x.startswith('outgroup_ldo_'), df.keys()))
    df['outgroup_species'] = df.apply(lambda x: closest_map.get((x.sp_tree_tail, x.sp_tree_head), None), axis=1)
    df['out_gene'] = df.apply(lambda x: x['outgroup_ldo_{}'.format(x.outgroup_species)] if x.outgroup_species is not None else None, axis=1)

    return df.drop(columns=outgroup_keys)

Generate expression data outgroup choices

In [7]:
expr_test_fn = './results/outgroup_tests_expression.tsv.gz'
if not os.path.isfile(expr_test_fn):
    pairs_df = pd.read_csv('./results/outgroup_tests.tsv.gz', sep='\t', low_memory=False)
    pairs = {sp: zdf for (sp, zdf) in pairs_df[pairs_df.species.isin(dataset['animals'] | dataset['plants'])].groupby('species')}
    
    z = []
    for sp in tqdm(dataset['animals']):
        x = generate_candidates(sp, pairs[sp], possible_species=dataset['animals'])
        z.append(filter_to_closest(pairs[sp], x))
    
    for sp in tqdm(dataset['plants']):
        x = generate_candidates(sp, pairs[sp], possible_species=dataset['plants'])
        z.append(filter_to_closest(pairs[sp], x))
    
    zdf = pd.concat(z)
    zdf.to_csv(expr_test_fn, sep='\t', index=False)

Generate choices for outgroup structural alphafold tests

In [8]:
struct_test_fn = './results/outgroup_tests_structure.tsv.gz'
if not os.path.isfile(struct_test_fn):
    pairs_df = pd.read_csv('./results/outgroup_tests.tsv.gz', sep='\t', low_memory=False)
    
    z = []
    for (sp, zdf) in tqdm(pairs_df.groupby('species')):
        x = generate_candidates(sp, zdf, possible_species=None)
        z.append(filter_to_closest(zdf, x))
    
    zdf1 = pd.concat(z)
    zdf1.to_csv(struct_test_fn, sep='\t', index=False)

---

Functions to load the data

In [9]:
EXPR = './results/expr'
TAU = './results/tau'
    

def load_expr(sp, expr_fn, tau_fn, map_fn, genes=None):
    # load the expression data, into a consistent format.
    map_df = pd.read_hdf(map_fn, sp)

    if genes is not None:
        map_df = map_df[map_df['UniProtKB'].isin(genes)]

    # load expression data
    expr_df = pd.read_hdf(expr_fn, sp)

    # temporarily flatten
    mi = expr_df.columns.copy()
    expr_df.columns = list(range(len(expr_df.columns)))
    
    # merge and apply transformation
    expr_df = pd.merge(expr_df, map_df, left_on='gene_id', right_on='xref').set_index('UniProtKB')[expr_df.columns]
    expr_df = expr_df.apply(np.arcsinh)

    # regroup the tissue names
    expr_df.columns = mi

    # average on columns!
    def avg():
        for k in set(map(lambda x: x[0], expr_df.columns)):
            yield pd.DataFrame({k: expr_df.loc[:,k].mean(axis=1)})

    expr_df = pd.concat(avg(), axis=1)

    # load the tau values
    tau_df = pd.read_hdf(tau_fn, sp)
    tau = pd.merge(pd.DataFrame({'tau': tau_df}), map_df, left_on='gene_id', right_on='xref').set_index('UniProtKB')['tau'].to_dict()

    return (expr_df, tau)


def compute_expr_sim(pair_df, expr_df, sim_func, key1, key2):
    return pair_df.p_apply(lambda x: sim_func(expr_df, x[key1], x[key2]), axis=1)

def compute_pearson_(expr_df, x, y):
    if x in expr_df.index and y in expr_df.index:
        e_x = expr_df.loc[x].values
        e_y = expr_df.loc[y].values
        if np.all(e_x == e_x[0]) or np.all(e_y == e_y[0]):
            return (None, None)
        return pearsonr(e_x, e_y)
    else:
        return (None, None)

---

## Run for each species

In [10]:
databases = {'animals': [('animal_categorised_tpm_expr.h5', 'animal_categorised_tau.h5', 'animal_categorised'),
                         ('animal_maincat_tpm_expr.h5', 'animal_maincat_tau.h5', 'animal_maincat'),
                         ('animal_tpm_expr.h5', 'animal_alldata_tau.h5', 'animal_alldata')],
             'plants': [('plant_general_tpm_expr.h5', 'plant_general_tau.h5', 'plant_general'),
                        ('plant_specific_tpm_expr.h5', 'plant_specific_tau.h5', 'plant_specific')]}

In [11]:
RES_PATH = './results/outgroup_correlation'
!mkdir -p {RES_PATH}


pairs_df = pd.read_csv('./results/outgroup_tests_expression.tsv.gz', sep='\t', low_memory=False)
pairs = {sp: zdf for (sp, zdf) in pairs_df[pairs_df.species.isin(dataset['animals'] | dataset['plants'])].groupby('species')}


for (sp, zdf) in tqdm(pairs.items()):
    if sp in dataset['animals']:
        db_key = 'animals'
    elif sp in dataset['plants']:
        db_key = 'plants'
    else:
        raise ValueError('unknown species {}'.format(sp))

    genes_to_load = (set(zdf.ldo_gene) | set(zdf.mdo_gene))

    # run for each database
    for (expr_fn, tau_fn, key) in databases[db_key]:
        (expr_df, tau_expr) = load_expr(sp,
                                        expr_fn=f'{EXPR}/{expr_fn}',
                                        tau_fn=f'{TAU}/{tau_fn}',
                                        map_fn=f'{EXPR}/gene_map.h5',
                                        genes=genes_to_load)
        tmp = []
        for (out_sp, zzdf) in zdf.groupby('outgroup_species'):
            (out_expr_df, out_tau_expr) = load_expr(out_sp,
                                                    expr_fn=f'{EXPR}/{expr_fn}',
                                                    tau_fn=f'{TAU}/{tau_fn}',
                                                    map_fn=f'{EXPR}/gene_map.h5',
                                                    genes=set(zzdf.out_gene))
            # comparable expression
            header = list(set(expr_df.keys()) & set(out_expr_df.keys()))
            if len(header) == 0:
                continue
                
            expr_shared_df = pd.concat((expr_df[header], out_expr_df[header]))
            
            # compute within-species correlation on comparable tissues
            pcc = compute_expr_sim(zzdf, expr_shared_df, compute_pearson_, key1='ldo_gene', key2='mdo_gene')
            zzdf['pcc_stat'] = pcc.apply(lambda x: x[0])
            zzdf['pcc_p'] = pcc.apply(lambda x: x[1])

            pcc = compute_expr_sim(zzdf, expr_shared_df, compute_pearson_, key1='ldo_gene', key2='out_gene')
            zzdf['out_ldo_pcc_stat'] = pcc.apply(lambda x: x[0])
            zzdf['out_ldo_pcc_p'] = pcc.apply(lambda x: x[1])

            pcc = compute_expr_sim(zzdf, expr_shared_df, compute_pearson_, key1='mdo_gene', key2='out_gene')
            zzdf['out_mdo_pcc_stat'] = pcc.apply(lambda x: x[0])
            zzdf['out_mdo_pcc_p'] = pcc.apply(lambda x: x[1])
    
            zzdf['ldo_tau'] = zzdf['ldo_gene'].apply(lambda x: tau_expr.get(x, None))
            zzdf['mdo_tau'] = zzdf['mdo_gene'].apply(lambda x: tau_expr.get(x, None))
            zzdf['out_tau'] = zzdf['out_gene'].apply(lambda x: out_tau_expr.get(x, None))

            tmp.append(zzdf)
            
        if len(tmp) > 0:
            pd.concat(tmp).to_hdf(f'{RES_PATH}/{key}.h5', sp, complevel=9, complib='zlib')

  0%|          | 0/36 [00:00<?, ?it/s]

<LAMBDA> DONE:   0%|          | 0/199 [00:00<?, ?it/s]

<LAMBDA> DONE:   0%|          | 0/199 [00:00<?, ?it/s]

<LAMBDA> DONE:   0%|          | 0/199 [00:00<?, ?it/s]

<LAMBDA> DONE:   0%|          | 0/207 [00:00<?, ?it/s]

...

<LAMBDA> DONE:   0%|          | 0/409 [00:00<?, ?it/s]

<LAMBDA> DONE:   0%|          | 0/61 [00:00<?, ?it/s]

<LAMBDA> DONE:   0%|          | 0/61 [00:00<?, ?it/s]

<LAMBDA> DONE:   0%|          | 0/61 [00:00<?, ?it/s]

/tmp/ipykernel_3805226/2830027787.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(tmp).to_hdf(f'{RES_PATH}/{key}.h5', sp, complevel=9, complib='zlib')
/tmp/ipykernel_3805226/2830027787.py:60: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['sp_tree_tail', 'sp_tree_head', 'ldo_category', 'mdo_category',
       'species', 'ldo_gene', 'mdo_gene', 'outgroup_species', 'out_gene'],
      dtype='object')]

  pd.concat(tmp).to_hdf(f'{RES_PATH}/{key}.h5', sp, complevel=9, complib='zlib')


In [13]:
pairs_df.head()

,fam_id,fam_tail_idx,sp_tree_edge,sp_tree_tail,sp_tree_head,ldo_fam_head_idx,mdo_fam_head_idx,ldo_branch_length,mdo_branch_length,expected_branch_length,...,ldo_delta_exp_p,mdo_delta_exp_p,ldo_category,mdo_category,multifurcation_idx,species,ldo_gene,mdo_gene,outgroup_species,out_gene
0,10003,548,241,Sauria,CHICK,551,550,0.227,0.269,0.125,...,0.564241,0.417161,normal,normal,0,CHICK,P80566,F1N8Q1,NaN,NaN
1,10005,2,209,Craniata-Tunicata,Euteleostomi,4,41,0.604,0.677,0.353,...,0.440508,0.293490,normal,normal,0,CHICK,F1P0N0,E1BSJ7,NaN,NaN
2,10010,20,209,Craniata-Tunicata,Euteleostomi,81,22,0.152,0.179,0.316,...,0.414690,0.476206,normal,normal,0,CHICK,A0A3S5ZPQ9,A0A1D5NWQ4,NaN,NaN
3,10015,183,241,Sauria,CHICK,185,186,0.017,0.017,0.135,...,0.513520,0.513520,normal,normal,0,CHICK,A0A1D5PY09,P38529,NaN,NaN
4,10015,224,241,Sauria,CHICK,226,227,0.026,0.026,0.135,...,0.546475,0.546475,normal,normal,0,CHICK,A0A3Q2U082,P38530,NaN,NaN
